In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
X = tf.placeholder(dtype=tf.float32, shape = ((None, 100, 100, 3)))
Y = tf.placeholder(dtype=tf.float32, shape = (None, 1))

In [3]:
Z1 = tf.nn.conv2d(input = X, filter = tf.Variable(initial_value=tf.random_normal(shape = [3, 3, 3, 32])), strides=[1, 2, 2, 1], padding="SAME")
A1 = tf.nn.relu(Z1)

Z2 = tf.nn.conv2d(input= A1, filter = tf.Variable(initial_value=tf.random_normal(shape = [3, 3, 32, 64])), strides=[1, 2, 2, 1], padding="VALID")
A2 = tf.nn.relu(Z2)

Z3 = tf.nn.conv2d(input= A2, filter = tf.Variable(initial_value=tf.random_normal(shape = [3, 3, 64, 128])), strides=[1, 2, 2, 1], padding="SAME")
A3 = tf.nn.relu(Z3)

Z4 = tf.nn.conv2d(input= A3, filter = tf.Variable(initial_value=tf.random_normal(shape = [3, 3, 128, 256])), strides=[1, 2, 2, 1], padding="VALID")
A4 = tf.nn.relu(Z4)

Z5 = tf.nn.conv2d(input= A4, filter = tf.Variable(initial_value=tf.random_normal(shape = [3, 3, 256, 512])), strides=[1, 2, 2, 1], padding="SAME")
A5 = tf.nn.relu(Z5)

Z6 = tf.nn.conv2d(input= A5, filter = tf.Variable(initial_value=tf.random_normal(shape = [3, 3, 512, 1024])), strides=[1, 2, 2, 1], padding="VALID")
A6 = tf.nn.relu(Z6)

In [4]:
F = tf.contrib.layers.flatten(inputs= A6)

In [5]:
Z7 = tf.matmul(F, tf.Variable(initial_value= tf.random_normal(shape = [1024, 512]))) + tf.Variable(initial_value= tf.random_normal(shape = [1, 512]))
A7 = tf.nn.relu(Z7)

Z8 = tf.matmul(A7, tf.Variable(initial_value= tf.random_normal(shape = [512, 256]))) + tf.Variable(initial_value= tf.random_normal(shape = [1, 256]))
A8 = tf.nn.relu(Z8)

Z9 = tf.matmul(A8, tf.Variable(initial_value= tf.random_normal(shape = [256, 128]))) + tf.Variable(initial_value= tf.random_normal(shape = [1, 128]))
A9 = tf.nn.relu(Z9)

In [6]:
def loss(Y, A9):
    return tf.maximum(tf.reduce_mean(tf.squared_difference(A9[0], A9[1]) - tf.constant(value = 0.7)), 0)

In [7]:
LOSS = loss(Y, A9)

In [8]:
opt = tf.train.AdamOptimizer(learning_rate= 0.01)
train_ = opt.minimize(LOSS)